In [2]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer

import logging

logging.getLogger().setLevel(logging.INFO)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
!python --version

Python 3.8.5


In [4]:
!pip show amazon-braket-schemas
!pip show amazon-braket-sdk
!pip show amazon-braket-ocean-plugin

Name: amazon-braket-schemas
Version: 1.5.0
Summary: An open source library that contains the schemas for Amazon Braket
Home-page: https://github.com/aws/amazon-braket-schemas-python
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /Users/yonmzn/opt/anaconda3/lib/python3.8/site-packages
Requires: pydantic
Required-by: amazon-braket-default-simulator
Name: amazon-braket-sdk
Version: 1.11.0
Summary: An open source library for interacting with quantum computing devices on Amazon Braket
Home-page: https://github.com/aws/amazon-braket-sdk-python
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /Users/yonmzn/opt/anaconda3/lib/python3.8/site-packages
Requires: amazon-braket-default-simulator, backoff, boltons, boto3, coverage, nest-asyncio, networkx, numpy
Required-by: amazon-braket-ocean-plugin
Name: amazon-braket-ocean-plugin
Version: 1.0.7
Summary: An open source framework for interacting with D-Wave's Ocean library through Ama

In [25]:
# parameters shared by different steps
qa_task_id = "c3d75163-222e-451b-898f-705956b068e6"
s3_bucket = f"amazon-braket-1a222675c751" # the name of the bucket
prefix = "annealer-experiment" # the name of the folder in the bucket

# raw_path = './molecule-data/Aspirin.mol2'
raw_path = './molecule-data/117_ideal.mol2'
data_path = './qmu_117_ideal_data_latest.pickle'
model_path = './qmu_117_ideal_model_latest.pickle'

# Prepare Molecule Data

In [5]:
mol_data = MoleculeData(raw_path, 'qmu')

mol_data.save("latest")

INFO:root:parse mol2 file!
INFO:root:finish save qmu_117_ideal_data_latest.pickle


'./qmu_117_ideal_data_latest.pickle'

# Build QUBO model for molecule unfolding

In [5]:
init_param = {}
method = ['pre-calc']

for mt in method:
    if mt == 'pre-calc':
        init_param[mt] = {}
        init_param[mt]['param'] = ['M', 'D', 'A', 'hubo_qubo_val']
    
qmu_qubo = QMUQUBO(mol_data, method, **init_param)

NameError: name 'mol_data' is not defined

In [7]:
model_param = {}
# parameters
num_rotation_bond = mol_data.bond_graph.rb_num

method = 'pre-calc'
model_param[method] = {}
# model_param[method]['M'] = range(1, num_rotation_bond+1)
model_param[method]['M'] = [1]
model_param[method]['D'] = [4]
model_param[method]['A'] = [300]
model_param[method]['hubo_qubo_val'] = [200]

qmu_qubo.build_model(**model_param)


INFO:root:['30+31']
INFO:root:Construct model for M:1,D:4,A:300,hubo_qubo_val:200 0.0002522865931193034 min


0

In [8]:
model_info = qmu_qubo.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {1}
INFO:root:param: D, value {4}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


In [9]:
qmu_qubo.save("latest")

INFO:root:finish save qmu_117_ideal_model_latest.pickle


'./qmu_117_ideal_model_latest.pickle'

# Optimize to find the configuration

In [5]:
from utility.AnnealerOptimizer import Annealer
from utility.QMUQUBO import QMUQUBO
import logging

In [6]:
qmu_qubo_optimize = QMUQUBO.load(model_path)


In [7]:
model_info = qmu_qubo_optimize.describe_model()

In [8]:
M = 1
D = 4
A = 300
hubo_qubo_val = 200
model_name = "{}_{}_{}_{}".format(M, D, A, hubo_qubo_val)
method = "pre-calc"

qubo_model = qmu_qubo_optimize.get_model(method, model_name)

In [9]:
method = 'dwave-sa'

optimizer_param = {}
optimizer_param['shots'] = 1

sa_optimizer = Annealer(qubo_model, method, **optimizer_param)

use simulated annealer from dimod


In [10]:
sa_optimize_result = sa_optimizer.fit()


fit() ...
fit result.model_info={'model_name': '1_4_300_200', 'rb_name': ['30+31'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x_8_3', '4': 'x_8_4'}, '9': {'1': 'x_9_1', '2': 'x_9_2', '3': 'x_9_3', '4': 'x_9_4'}, '10': {'1': 'x_10_1', '2': 'x_10_2', '3': 'x_10_3', '4': 'x_10_4'}, '11': {'1': 'x_11_1', '2': 'x_11_2', '3': 'x_11_3', '4': 'x_11_4'}, '12': {'1': 'x_12_1', '2': 'x_12_2', '3': 'x_12_3', '4': 'x_12_4'}, '13': {'1': 'x_13_1', '2': 'x_13_2', '3': 'x_13_3', '4': 'x_13_4'}, '14': {'1': 'x_14_1', '2': 'x_14_2', '3': 'x_14_3', '4': 'x_14_4'}, '

In [11]:
method = 'dwave-qa'

optimizer_param = {}
optimizer_param['shots'] = 1
optimizer_param['bucket'] = s3_bucket # the name of the bucket
optimizer_param['prefix'] = prefix # the name of the folder in the bucket
optimizer_param['device'] = "arn:aws:braket:::device/qpu/d-wave/Advantage_system4"
optimizer_param["embed_method"] = "default"

qa_optimizer = Annealer(qubo_model, method, **optimizer_param)

use quantum annealer arn:aws:braket:::device/qpu/d-wave/Advantage_system4 


In [12]:
# not create annealing task, only embedding logic
qa_optimizer.embed()
# create annealing task
qa_optimize_result = qa_optimizer.fit()

fit() ...
fit result.model_info={'model_name': '1_4_300_200', 'rb_name': ['30+31'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x_8_3', '4': 'x_8_4'}, '9': {'1': 'x_9_1', '2': 'x_9_2', '3': 'x_9_3', '4': 'x_9_4'}, '10': {'1': 'x_10_1', '2': 'x_10_2', '3': 'x_10_3', '4': 'x_10_4'}, '11': {'1': 'x_11_1', '2': 'x_11_2', '3': 'x_11_3', '4': 'x_11_4'}, '12': {'1': 'x_12_1', '2': 'x_12_2', '3': 'x_12_3', '4': 'x_12_4'}, '13': {'1': 'x_13_1', '2': 'x_13_2', '3': 'x_13_3', '4': 'x_13_4'}, '14': {'1': 'x_14_1', '2': 'x_14_2', '3': 'x_14_3', '4': 'x_14_4'}, '

In [20]:
qa_task_id = qa_optimizer.get_task_id()
logging.info(f"task id is {qa_task_id}")

INFO:root:task id is c3d75163-222e-451b-898f-705956b068e6


In [21]:
logging.info(f"dwave-sa run time {sa_optimize_result['time']}")
logging.info(f"dwave-qa run time {qa_optimize_result['time']}")

INFO:root:dwave-sa run time 0.02351999282836914
INFO:root:dwave-qa run time 5.408229112625122


# Postprocess for evaluation

#### get time for qpu

In [26]:
from utility.ResultProcess import ResultParser
import time
import logging

timestamp = time.strftime("%Y%m%d-%H")

method = "dwave-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
logging.debug(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

logging.info(f"time for {method}: \n \
    local time is {local_time}")

_load_raw_result
load simulated annealer raw result
MoleculeData.load()
_init_mol_file
_parse_model_info
_parse_model_info() model_info = {'model_name': '1_4_300_200', 'rb_name': ['30+31'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x_8_3', '4': 'x_8_4'}, '9': {'1': 'x_9_1', '2': 'x_9_2', '3': 'x_9_3', '4': 'x_9_4'}, '10': {'1': 'x_10_1', '2': 'x_10_2', '3': 'x_10_3', '4': 'x_10_4'}, '11': {'1': 'x_11_1', '2': 'x_11_2', '3': 'x_11_3', '4': 'x_11_4'}, '12': {'1': 'x_12_1', '2': 'x_12_2', '3': 'x_12_3', '4': 'x_12_4'}, '13': {'1': 'x_13_1', '2': 'x_1

INFO:root:time for dwave-sa: 
     local time is 0.02351999282836914


parse simulated annealer result
sa only has local_time!


In [27]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_process_result.save_mol_file(f"{timestamp}")

INFO:root:!!!!!!!!!!!! found van der waals check fail at 9 and 67 with check: 2.9 v.s. real 2.7242024227285313
INFO:root:!!!!!!!!!!!! found van der waals check fail at 10 and 29 with check: 3.4 v.s. real 3.3487103323518443
INFO:root:!!!!!!!!!!!! found van der waals check fail at 10 and 64 with check: 2.9 v.s. real 2.535259872281341
INFO:root:!!!!!!!!!!!! found van der waals check fail at 14 and 67 with check: 2.9 v.s. real 2.669994202240896
INFO:root:!!!!!!!!!!!! found van der waals check fail at 29 and 49 with check: 2.9 v.s. real 2.649657957171076
INFO:root:!!!!!!!!!!!! found van der waals check fail at 37 and 66 with check: 2.9 v.s. real 2.6543531415393846
INFO:root:!!!!!!!!!!!! found van der waals check fail at 38 and 66 with check: 2.9 v.s. real 2.697112159328937


generate_optimize_pts()
generate_optimize_pts model_info={'model_name': '1_4_300_200', 'rb_name': ['30+31'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x_8_3', '4': 'x_8_4'}, '9': {'1': 'x_9_1', '2': 'x_9_2', '3': 'x_9_3', '4': 'x_9_4'}, '10': {'1': 'x_10_1', '2': 'x_10_2', '3': 'x_10_3', '4': 'x_10_4'}, '11': {'1': 'x_11_1', '2': 'x_11_2', '3': 'x_11_3', '4': 'x_11_4'}, '12': {'1': 'x_12_1', '2': 'x_12_2', '3': 'x_12_3', '4': 'x_12_4'}, '13': {'1': 'x_13_1', '2': 'x_13_2', '3': 'x_13_3', '4': 'x_13_4'}, '14': {'1': 'x_14_1', '2': 'x_14_2', '3': 'x

INFO:root:!!!!!!!!!!!! found van der waals check fail at 49 and 64 with check: 2.4 v.s. real 1.663679602567754
INFO:root:!!!!!!!!!!!! found van der waals check fail at 66 and 69 with check: 2.4 v.s. real 1.9756453629130912


save_mol_file 20211223-13
finish save ./molecule-data/117_ideal_dwave-sa_20211223-13.mol2 and ./molecule-data/117_ideal_dwave-sa_20211223-13.json


['./molecule-data/117_ideal_dwave-sa_20211223-13.mol2',
 './molecule-data/117_ideal_dwave-sa_20211223-13.json']

In [28]:
sa_process_result.parameters

{'volume': {'initial': 19926.786975170362,
  'optimize': 19752.58165516736,
  'gain': 0.9912577316041934}}

In [29]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
logging.debug(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

logging.info(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

_load_raw_result
load quantum annealer raw result
_read_result_obj
_read_result_obj: annealer-experiment/c3d75163-222e-451b-898f-705956b068e6/qa_result.pickle
MoleculeData.load()
_init_mol_file
_parse_model_info
_parse_model_info() model_info = {'model_name': '1_4_300_200', 'rb_name': ['30+31'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x_8_3', '4': 'x_8_4'}, '9': {'1': 'x_9_1', '2': 'x_9_2', '3': 'x_9_3', '4': 'x_9_4'}, '10': {'1': 'x_10_1', '2': 'x_10_2', '3': 'x_10_3', '4': 'x_10_4'}, '11': {'1': 'x_11_1', '2': 'x_11_2', '3': 'x_11_3', '4': 'x_

INFO:root:time for dwave-qa: 
     local time is 5.408229112625122,
     task time is 1.313, 
     qpu total time is 0.023824, 
     qpu access time is 0.008564


#### parse results

In [30]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_process_result.save_mol_file(f"{timestamp}")

INFO:root:!!!!!!!!!!!! found van der waals check fail at 9 and 67 with check: 2.9 v.s. real 2.7242024227285313
INFO:root:!!!!!!!!!!!! found van der waals check fail at 10 and 29 with check: 3.4 v.s. real 3.3487103323518443
INFO:root:!!!!!!!!!!!! found van der waals check fail at 10 and 64 with check: 2.9 v.s. real 2.535259872281341
INFO:root:!!!!!!!!!!!! found van der waals check fail at 14 and 67 with check: 2.9 v.s. real 2.669994202240896
INFO:root:!!!!!!!!!!!! found van der waals check fail at 29 and 49 with check: 2.9 v.s. real 2.649657957171076
INFO:root:!!!!!!!!!!!! found van der waals check fail at 37 and 66 with check: 2.9 v.s. real 2.6543531415393846
INFO:root:!!!!!!!!!!!! found van der waals check fail at 38 and 66 with check: 2.9 v.s. real 2.697112159328937


generate_optimize_pts()
generate_optimize_pts model_info={'model_name': '1_4_300_200', 'rb_name': ['30+31'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x_8_3', '4': 'x_8_4'}, '9': {'1': 'x_9_1', '2': 'x_9_2', '3': 'x_9_3', '4': 'x_9_4'}, '10': {'1': 'x_10_1', '2': 'x_10_2', '3': 'x_10_3', '4': 'x_10_4'}, '11': {'1': 'x_11_1', '2': 'x_11_2', '3': 'x_11_3', '4': 'x_11_4'}, '12': {'1': 'x_12_1', '2': 'x_12_2', '3': 'x_12_3', '4': 'x_12_4'}, '13': {'1': 'x_13_1', '2': 'x_13_2', '3': 'x_13_3', '4': 'x_13_4'}, '14': {'1': 'x_14_1', '2': 'x_14_2', '3': 'x

INFO:root:!!!!!!!!!!!! found van der waals check fail at 49 and 64 with check: 2.4 v.s. real 1.663679602567754
INFO:root:!!!!!!!!!!!! found van der waals check fail at 66 and 69 with check: 2.4 v.s. real 1.9756453629130912


save_mol_file 20211223-13
finish save ./molecule-data/117_ideal_dwave-qa_20211223-13.mol2 and ./molecule-data/117_ideal_dwave-qa_20211223-13.json


['./molecule-data/117_ideal_dwave-qa_20211223-13.mol2',
 './molecule-data/117_ideal_dwave-qa_20211223-13.json']

In [31]:
qa_process_result.parameters

{'volume': {'initial': 19926.786975170362,
  'optimize': 19752.58165516736,
  'gain': 0.9912577316041934}}

### draft place

In [82]:
type(device.properties.service.executionWindows[0].windowStartHour)

datetime.time

In [9]:
import numpy as np
a = [-0.9578,-2.0342,0.0079]
b = [0.3270,-2.5523,0.0594]
def calc_distance_between_pts(pts1, pts2):
    pts1_middle = np.array(tuple(list(np.mean(np.array(pts1), axis=0))))
    pts2_middle = np.array(tuple(list(np.mean(np.array(pts2), axis=0))))
    return np.linalg.norm(pts1_middle-pts2_middle)

calc_distance_between_pts([a],[b])

1.3862867308028306

In [80]:
import time
gmt = time.gmtime(time.time())

diff = device.properties.service.executionWindows[0].windowStartHour.hour*60*60-gmt.tm_hour*60*60-gmt.tm_min*60-gmt.tm_sec

hour = diff//3600

minute = (diff - hour*60*60)//60

sec = diff - hour*60*60 - minute*60

print("next available time {}:{}:{}".format(hour, minute, sec))

next available time 4:34:11


In [75]:
device.properties.service.executionWindows[0].windowStartHour

datetime.time(13, 0)

In [287]:
import datetime
device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
exec_day = device.properties.service.executionWindows[0].executionDay.value
window_start_time = device.properties.service.executionWindows[0].windowStartHour
timedelta_hour = datetime.timedelta(hours=window_start_time.hour, minutes=window_start_time.minute)
current_time = datetime.datetime.utcnow()

diff = 0
today_day = current_time.weekday()

if exec_day == 'Weekdays':
    if today_day > 4:
        diff = 7 - today_day
elif exec_day == 'Weekend':
    if today_day < 5:
        diff = 5 - today_day

timedelta = datetime.timedelta(diff)

start_time = datetime.datetime(current_time.year, current_time.month, current_time.day) + timedelta

start_time = start_time + timedelta_hour

In [288]:
current_time

datetime.datetime(2021, 11, 22, 1, 51, 42, 975369)

In [289]:
current_time.timestamp()*1000

1637545902975.369

In [259]:
delta = start_time - current_time

day = delta.days

hour = delta.seconds//3600

minute = (delta.seconds - hour*60*60)//60

sec = delta.seconds - hour*60*60 - minute*60

print("next available time {} days {}:{}:{}".format(day, hour, minute, sec))

next available time 0 days 4:55:2


In [304]:
window = device.properties.service.executionWindows[0]
print(window)
print(window.executionDay.value)
print(window.windowStartHour.minute)
print(window.windowEndHour)

executionDay=<ExecutionDay.WEEKDAYS: 'Weekdays'> windowStartHour=datetime.time(13, 0) windowEndHour=datetime.time(2, 0)
Weekdays
0
02:00:00


In [13]:
class AwsDeviceTest:
    def __init__(self, device):
        self.__properties__ = device.properties
        self._status = device.status
    @staticmethod
    def _splitWindow(window):
        start_hour = window.windowStartHour.hour
        start_min = window.windowStartHour.minute
        end_hour = window.windowEndHour.hour
        end_min = window.windowEndHour.minute
        
        return start_hour, start_min, end_hour, end_min
    
    def _createTimeRange(date, window, available_days):
        TWENTY_FOUR_HOURS_IN_MILLISECONDS = 86400000
        # hour/min string -> numbers
        start_hour, start_min, end_hour, end_min = AwsDeviceTest._splitWindow(window)
        
        time_ranges = []
        
        for day in available_days:
            # Set the start and end times to be on the same day as the current time, then shift them to there correct day using milliseconds offset
            # Shifting after converting to milliseconds is much easier than dealing with calendar dates 
            start_utc = datetime.datetime(date.year, date.month, date.day, start_hour, start_min, 0).timestamp()*1000
            end_utc = datetime.datetime(date.year, date.month, date.day, end_hour, end_min, 0).timestamp()*1000
            
            start_utc = start_utc + TWENTY_FOUR_HOURS_IN_MILLISECONDS * (day - date.weekday())
            end_utc = end_utc + TWENTY_FOUR_HOURS_IN_MILLISECONDS * (day - date.weekday())
            
            if start_utc > end_utc:
                # If the time wraps to the next day and the day is Sunday(6) we need to wrap to Monday(0) so zero out the current weekday 
                if (day == 6):
                    wrap_start_utc = wrap_start_utc + TWENTY_FOUR_HOURS_IN_MILLISECONDS * (day - 7 - date.weekday())
                    wrap_end_utc = wrap_end_utc + TWENTY_FOUR_HOURS_IN_MILLISECONDS * (day - 6 - date.weekday())
                    
                    time_ranges.insert(0, [wrap_start_utc, wrap_end_utc])
                end_utc = end_utc + TWENTY_FOUR_HOURS_IN_MILLISECONDS
            
            time_ranges.append([start_utc, end_utc])
        
        return time_ranges
    
    def _getTimeDifference(date, window, available_days):
        # We need to get the available windows in the current week and the week after incase the next available day is a week away.
        # For example, if the current time 6 pm on Friday and the range is every Friday from 1pm-5pm, then the next range will be a next Friday at 1pm
        # TODO

        # Get current time in milliseconds
        current_time = date.timestamp() * 1000
        # Create an array of possible available ranges
        time_ranges = AwsDeviceTest._createTimeRange(date, window, available_days)
        
        for time_range in time_ranges:
            # If the current time is in the range then return 0
            if current_time > time_range[0] and current_time < time_range[1]:
                return 0
            # The ranges will be in chronological order the current time will always be either in a range or less than one
            elif current_time < time_range[0]:
                return time_range[0] - current_time

    @staticmethod
    def next_available_time(self) -> int:
        """
        Get the next available time in seconds. If the device is offline or retired, 
        The return value is -1

        Examples:
            >>> next_time_in_seconds = AwsDevice.next_available_time()

        Returns:
            int : next available time in seconds
        """
        # refresh to get the recent meta data
        # self._populate_properties(self._aws_session)
        DaysEnum = {
            'Sunday': 6,
            'Monday': 0,
            'Tuesday': 1,
            'Wednesday': 2,
            'Thursday': 3,
            'Friday': 4,
            'Saturday': 5
            }
            
        if self._status in ['OFFLINE', 'RETIRED']:
            # the device is not online
            return -1
            
        windows = self._properties.service.executionWindows
        
        # minimum time is 20 ms
        min_time = None
        calc_time = 0
        
        date = datetime.datetime.utcnow()

        for window in windows:
            exec_day = window.executionDay.value
            # "Everyday" | "Weekdays" | "Weekend" | "Monday" | "Tuesday" | "Wednesday" | "Thursday" | "Friday" | "Saturday" | "Sunday"
            if exec_day == 'Everyday':
                calc_time = AwsDevice._getTimeDifference(date, window, range(0,7))
            elif exec_day == 'Weekdays':
                calc_time = AwsDevice._getTimeDifference(date, window, range(0,6))
            elif exec_day == 'Weekend':
                calc_time = AwsDevice._getTimeDifference(date, window, [5,6])
            else:
                calc_time = AwsDevice._getTimeDifference(date, window, [DaysEnum[exec_day]])
        
        if (min_time == None or calc_time < min_time):
            min_time = calc_time
        if min_time == None:
            return 0
        else:
            return min_time/1000.

In [14]:
from braket.aws import AwsDevice
import datetime
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/Advantage_system4")
# device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/Advantage_system1")
# device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
# device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
# device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-8")
# device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-9")
# device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-10")
# device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")
# device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/tn1")
# device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")
awsdevice = AwsDeviceTest(device)

min_time = awsdevice.next_available_time()

hour = min_time//3600

minute = (min_time - hour*60*60)//60

sec = min_time - hour*60*60 - minute*60

print("next available time {}:{}:{}".format(hour, minute, sec))


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


TypeError: next_available_time() missing 1 required positional argument: 'self'

In [17]:
awsdevice = AwsDeviceTest(device)
awsdevice.next_available_time()

TypeError: next_available_time() missing 1 required positional argument: 'self'

In [382]:
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6")
min_time = getNextAvailable(device)

hour = min_time//3600

minute = (min_time - hour*60*60)//60

sec = min_time - hour*60*60 - minute*60

print("next available time {}:{}:{}".format(hour, minute, sec))

calc time 0
next available time 0.0:0.0:0.0


In [366]:
device.properties.service.executionWindows[0]

DeviceExecutionWindow(executionDay=<ExecutionDay.WEEKDAYS: 'Weekdays'>, windowStartHour=datetime.time(13, 0), windowEndHour=datetime.time(2, 0))

In [346]:
date = datetime.datetime.utcnow()

In [352]:
start_hour = window.windowStartHour.hour
start_min = window.windowStartHour.minute
end_hour = window.windowEndHour.hour
end_min = window.windowEndHour.minute

start_utc = datetime.datetime(date.year, date.month, date.day, start_hour, start_min, 0)
end_utc = datetime.datetime(date.year, date.month, date.day, end_hour, end_min, 0)
start_utc.weekday()

0

In [257]:
today_day

4

In [102]:
import datetime

a = datetime.datetime.utcnow()
b = datetime.datetime(1970, 1, 1)
a -b

datetime.timedelta(days=18949, seconds=31654, microseconds=708438)

In [110]:
(today.weekday() + 6) % 7

4

In [109]:
today = datetime.datetime(2021, 11, 13)


offset = max(0, (today.weekday() + 6) % 7 - 3)


timedelta = datetime.timedelta(offset)


most_recent = today + timedelta


print(most_recent)

2021-11-14 00:00:00
